In [14]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from WickContractions.wick.contract import *

import os

from src.utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

## Operators

First we have to define the operators

In [16]:
from src.op_elementals import *

aOps=[Operator([MesonBaryonSource(1,{'flavor': ['u','u','u'], 'x': 'x_0', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['u','d'], 'x':'x_1', 'gamma': '\\Gamma^M'})])]

cOps=[Operator([MesonBaryonSink(1,{'flavor': ['u','u','u'], 'x': 'x_2', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['d','u'], 'x':'x_3', 'gamma': '\\Gamma^M'})])]

In [17]:
pprint(aOps[0])
pprint(cOps[0])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
diagrams=[]
for i,aOp in enumerate(aOps):
    for j,cOp in enumerate(cOps):
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)

print("Need to compute {} diagrams".format(len(diagrams)))
pprint("\\text{{diagram}}[0]={}".format(diagrams[0]))

Need to compute 24 diagrams


<IPython.core.display.Math object>

In [19]:
from WickContractions.laph.diagram import LDiagram

ldiags=[]
for d in diagrams:
    ldiags.append(LDiagram(d))
for d in ldiags:
    d.create_b_blocks()
    d.create_m_blocks()
    d.create_hadron_blocks()
    d.combine_indices()
    d.create_hadron_source()

In [20]:
pprint(ldiags[0])

<IPython.core.display.Math object>

In [21]:
ldiags[0].as_graph()

{'B(x_0,t_i,\\Gamma^B)|B^*(x_2,t_i,t_f,\\Gamma^B)': [[2, 0], [0, 2]],
 'B(x_0,t_i,\\Gamma^B)|M(x_3,t_i,t_f,\\Gamma^M)': [[1, 1]],
 'B^*(x_2,t_i,t_f,\\Gamma^B)|M(x_1,t_i,\\Gamma^M)': [[1, 0]],
 'M(x_1,t_i,\\Gamma^M)|M(x_3,t_i,t_f,\\Gamma^M)': [[1, 0]]}

## uuu (ubd) -> bububu (bu d)

$$
    d0 = B_{012}M_{23}B_{014}M_{43} \\
    d1 = B_{012}M_{34}B_{012}M_{34}.
$$

You should compute 

$$
A_{23} = B_{012}B_{013}
$$

Then

$$
    d0 = A_{24}M_{23}M_{43} \\ 
    d1 = A_{22}M_{34}M_{34}
$$